Informática - 1º de Física - UMU
<br>
**Computación Científica**
<br>
<p style="color:#808080"> <small><small>
9/12/2019
</small></small></p>

## Partículas en una caja

Para simular sistemas en los que casi todo el tiempo los objetos se mueven a velocidad constante, o donde interesa estudiar el comportamiento global sin preocuparnos mucho de los detalles, no es necesario un integrador preciso y podemos usar el sencillo método de Euler:

$$ v = \frac{dr}{dt} \simeq \frac{\Delta r}{\Delta t} $$

Para un $\Delta t$ pequeño:

$$r_{k+1} = r_{k} + v_k \Delta t $$

Gráficamente:


<div style='width:35%;  margin:auto; margin-top:1em'>
![euler1.png](attachment:euler1.png)
</div>

Si la velocidad es constante el objeto seguirá una trayectoria recta. Si hay aceleración, el método de Euler se aplica en dos pasos: uno para actualizar la velocidad y otro para la posición.



<div style="width:40%; margin:auto; margin-top:1em">
![euler2.png](attachment:euler2.png)
</div>

\begin{aligned}
v_{k+1} &= v_{k} + a_k \Delta t \\
r_{k+1} &= r_{k} + v_{k+1} \Delta t 
\end{aligned}

Marcamos en rojo la velocidad para indicar que es la ya corregida por la aceleración en ese paso.

Vamos a ver algunas simulaciones mediante esta técnica mostrando el resultado mediante animaciones.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from matplotlib import animation, rc
rc('animation', html='html5')

Partícula en movimiento uniforme

In [2]:
# preparamos el gráfico
fig, ax = plt.subplots(figsize=(6,6))
ax.set_xlim(( 0,1 )); plt.xlabel('x')
ax.set_ylim(( 0, 1)); plt.ylabel('y')
[punto] = ax.plot([],[],'.',color='black',markerSize=20)
plt.close();

# condiciones iniciales
r = np.array([0.2, 0.2])
v = np.array([0.003, 0.007])

# intervalo de integración
dt = 1

def animate(k):
    global r, v
    
    # simulación física
    r = r + v * dt
    
    # nuevo fotograma
    punto.set_data(r[0],r[1])
    return ()

animation.FuncAnimation(fig, animate, init_func=lambda:[],frames=5*30, interval=1000/30, blit=True)

Partícula con aceleración

In [3]:
# preparamos el gráfico
fig, ax = plt.subplots(figsize=(6,6))
ax.set_xlim(( 0,1 )); plt.xlabel('x')
ax.set_ylim(( 0, 1)); plt.ylabel('y')
[punto] = ax.plot([],[],'.',color='black',markerSize=20)
plt.close();

# condiciones iniciales
r = np.array([0.2, 0.2])
v = np.array([0.006, 0.02])

# intervalo de integración
dt = 1

def animate(k):
    global r, v
    
    # simulación física
    a = np.array([0, -0.0005])
    
    v += a * dt
    r += v * dt
    
    # nuevo fotograma
    punto.set_data(r[0],r[1])
    return ()

animation.FuncAnimation(fig, animate, init_func=lambda:[],frames=5*30, interval=1000/30, blit=True)

Rebote contra el suelo. Cuando la partícula se sale de los límites hay simular el rebote, lo que implica deshacer el movimiento y cambiar el sentido de la velocidad.

In [4]:
# preparamos el gráfico
fig, ax = plt.subplots(figsize=(6,6))
ax.set_xlim(( 0,1 )); plt.xlabel('x')
ax.set_ylim(( 0, 1)); plt.ylabel('y')
[punto] = ax.plot([],[],'.',color='black',markerSize=20)
plt.close();

# condiciones iniciales
r = np.array([0.2, 0.2])
v = np.array([0.006, 0.02])

# intervalo de integración
dt = 1

def animate(k):
    global r, v
    
    # simulación física
    
    a = np.array([0, -0.0005])
    
    v += a * dt
    r += v * dt
    
    if r[1] < 0:
        r[1] = r[1] - 2*v[1]*dt
        v[1] = -v[1]
    
    # nuevo fotograma
    punto.set_data(r[0],r[1])
    return ()

animation.FuncAnimation(fig, animate, init_func=lambda:[],frames=5*30, interval=1000/30, blit=True)

Para hacer la animación en "tiempo real" hay que ajustar las constantes físicas y usar un $\Delta t$ que corresponda a los frames por segundo con los que se genera el vídeo.

Añadimos un "rastro" para observar mejor la trayectoria de la partícula.

In [5]:
# preparamos el gráfico
fig, ax = plt.subplots(figsize=(6,6))
ax.set_xlim(( 0,1 )); plt.xlabel('x')
ax.set_ylim(( 0, 1)); plt.ylabel('y')
[rastro] = ax.plot([],[],'-',color='gray')
[punto] = ax.plot([],[],'.',color='black',markerSize=20)
plt.close();

from collections import deque

rs = deque(maxlen=20)

# condiciones iniciales
r = np.array([0.2, 0.2])
v = np.array([0.006, 0.02])

# intervalo de integración
dt = 1

def animate(k):
    global r, v
        
    a = np.array([0, -0.0005])
    
    v += a * dt
    r += v * dt
    
    if r[1] < 0:
        r[1] = r[1] - 2*v[1]*dt
        v[1] = -v[1]

    rs.append(r.copy()) # ojo
    
    # nuevo fotograma
    punto.set_data(r[0],r[1])
    rastro.set_data(*np.array(rs).T)   

    return ()

animation.FuncAnimation(fig, animate, init_func=lambda:[],frames=5*30, interval=1000/30, blit=True)

La simulación de múltiples partículas es prácticamente automática usando arrays.

In [6]:
fig, ax = plt.subplots(figsize=(6,6))
ax.set_xlim(( 0,1 )); plt.xlabel('x')
ax.set_ylim(( 0, 1)); plt.ylabel('y')
[puntos] = ax.plot([],[],'.',color='black',markerSize=8)
plt.close();


n = 1000

# posiciones y velocidades iniciales
r = 0.2 + 0.3  * np.random.rand(n,2)
v = 0.01 * np.random.randn(n,2)

dt = 1

def animate(k):
    global r, v

    r += v * dt
    
    puntos.set_data(r[:,0],r[:,1])
    return ()

animation.FuncAnimation(fig, animate, init_func=lambda:[],frames=5*30, interval=1000/30, blit=True)

En el siguiente ejemplo simulamos rebotes contra las paredes de la caja. Gracias a `numpy`, en vez de buscar explícitamente con `if` y `for` las partículas que se salen de los límites podemos usar una "máscara" (un array de condiciones lógicas) `reb` que indicará las componentes de velocidad que deben cambiar de sentido. Esto se realiza mediante la sencilla expresión `reb = (r > 1) |  (r < 0)`, cuyo resultado es un array $n\times2$ de condiciones lógicas que indican la partícula - coordenada que sufre un rebote. Este array se puede usar como índice de `r` y `v` para modificar solo los elementos deseados.

In [7]:
fig, ax = plt.subplots(figsize=(6,6))
ax.set_xlim(( 0,1 )); plt.xlabel('x')
ax.set_ylim(( 0, 1)); plt.ylabel('y')
[puntos] = ax.plot([],[],'.',color='black',markerSize=8)
plt.close();


n = 1000

r = 0.2 + 0.3  * np.random.rand(n,2)
v = 0.01 * np.random.randn(n,2)

dt = 1

def animate(k):
    global r, v
    
    r += v * dt
    
    reb = (r > 1) |  (r < 0)
    r[reb] -= 2*v[reb]*dt
    v[reb] = -v[reb]
    
    puntos.set_data(*r.T)
    return ()

animation.FuncAnimation(fig, animate, init_func=lambda:[], frames=5*30, interval=1000/30, blit=True)

Finalmente, con una modificación muy simple podemos simular la aceleración de la gravedad y choques inelásticos contra las paredes. En este caso usamos `scatter` para poder establecer cómodamente colores distintos en cada partícula.

In [8]:
n = 100

fig, ax = plt.subplots(figsize=(6,6))
ax.set_xlim(( 0,1 )); plt.xlabel('x')
ax.set_ylim(( 0, 1)); plt.ylabel('y')
puntos = ax.scatter([],[], s=80, alpha=0.8, c = np.random.random((n, 3)))
plt.close();


r = 0.4 + 0.3  * np.random.rand(n,2)
v = 0.01 * np.random.randn(n,2)

a = np.array([ 0, -0.001 ])

dt = 1

def animate(k):
    global r, v

    v += a * dt
    r += v * dt
    
    reb = (r > 1) |  (r < 0)
    r[reb] -= 2*v[reb]*dt
    v[reb] *= -0.8
    
    
    puntos.set_offsets(r)
    return ()

animation.FuncAnimation(fig, animate, init_func=lambda:[], frames=10*30, interval=1000/30, blit=True)

Las simulaciones anteriores no son exactas porque el rebote no se produce exactamente contra la pared, sino a una pequeña distancia, pero para observar el efecto global no merece la pena complicarlo. 

Como ejercicio, intenta reproducir [este vídeo](https://robot.inf.um.es/material/inforfis/graph/caja.mp4), en el que se muestra lo que ocurre cuando situamos una barrera dentro de la caja.

Para simular [choques entre partículas](https://robot.inf.um.es/material/inforfis/graph/billar.mp4) es necesario determinar en cada momento las parejas de partículas que entrarían en colisión y para cada una de ellas cambiar la velocidad en la componente perpendicular al plano tangente, conservando energía y momento. En un capítulo posterior se muestran varias simulaciones basadas en esta idea.